<a href="https://colab.research.google.com/github/Kevlarth/Alura-e-Google-imersao-IA/blob/main/Projeto_Alura_%2B_Google_Imers%C3%A3o_Ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Similaridade com Gemini Pro e Extração de Trechos Relevantes no Google Colab
&nbsp;&nbsp;&nbsp;&nbsp;Este notebook utiliza o poder do Gemini Pro para calcular a similaridade entre um assunto e um artigo em PDF, além de extrair os trechos mais relevantes do artigo.

## Passo 1: Instalação das bibliotecas

In [ ]:
!pip install -q -U PyPDF2 beautifulsoup4 nltk scikit-learn google-generativeai

## Passo 2: Importação das bibliotecas

In [ ]:
import requests
import PyPDF2
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import google.generativeai as genai
from google.colab import userdata

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Passo 3: Configuração da API Key do Gemini Pro
&nbsp;&nbsp;&nbsp;&nbsp;Para manter a chave de API segura, usaremos o userdata do Colab.


In [ ]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)


## Passo 4: Configuando o Model

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

generation_config = {
    "candidate_count": 1,
    "temperature": 0.7
}

safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE"
}

model = genai.GenerativeModel(model_name='models/gemini-1.5-pro-latest',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


## Passo 5: Funções auxiliares

In [ ]:
# Função para extrair texto de um PDF
def extract_text_from_pdf(pdf_url):
  response = requests.get(pdf_url)
  with open('temp.pdf', 'wb') as f:
    f.write(response.content)
  pdf_reader = PyPDF2.PdfReader('temp.pdf')
  text = ""
  for page in pdf_reader.pages:
    text += page.extract_text()
  return text

# Função para pré-processar o texto
def preprocess_text(text):
  text = text.lower()
  text = BeautifulSoup(text, "html.parser").get_text()
  text = " ".join(nltk.word_tokenize(text))
  return text

# Função para gerar o resumo do artigo com Gemini Pro
def summarize_with_gemini(text):
    response = model.generate_content(f'Por favor, gere um resumo conciso e informativo do seguinte artigo:\n\n{text}')
    summary = response.text
    return summary

# Função para calcular a similaridade lexical
def calculate_lexical_similarity(text1, text2):
  vectorizer = TfidfVectorizer()
  tfidf_matrix = vectorizer.fit_transform([text1, text2])
  similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
  return similarity

# Função para identificar trechos relevantes
def get_relevant_sentences(text, assunto, similarity_threshold=0.25):
  sentences = nltk.sent_tokenize(text)
  relevant_sentences = []
  for sentence in sentences:
    similarity = calculate_lexical_similarity(assunto, sentence)
    if similarity >= similarity_threshold:
      relevant_sentences.append(sentence)
  return relevant_sentences

## Passo 6: Utilização
&nbsp;&nbsp;&nbsp;&nbsp;Agora você pode usar a função analisar_similaridade_pdf para analisar seus PDFs!

In [ ]:
# Entrada do usuário
assunto = input("Digite o texto do assunto: ")
pdf_url = input("Digite a URL do artigo em PDF: ")

# Processamento
text = extract_text_from_pdf(pdf_url)
processed_text = preprocess_text(text)
processed_assunto = preprocess_text(assunto)
article_summary = summarize_with_gemini(processed_text)
similarity = calculate_lexical_similarity(processed_assunto, article_summary)
relevant_sentences = get_relevant_sentences(text, assunto)

# Saída
print(f"\nSimilaridade: {similarity:.2f}")
if relevant_sentences:
    print("\nTrechos relevantes:")
    for sentence in relevant_sentences:
        print(f"- {sentence}")